In [44]:
pip install langchain_openai langchain_community openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 35.3 MB/s eta 0:00:00


In [91]:
from kaggle_secrets import UserSecretsClient
from langchain_openai import OpenAI
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStore
from langchain.chains import RetrievalQA
from pathlib import Path
import os

In [ ]:
os.environ['OPENAI_API_KEY'] = UserSecretsClient().get_secret('OPENAI_API_KEY')

In [99]:
directory_in_str = '/content/text-files/'

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    length_function=len
)

all_documents = []
pathlist = Path(directory_in_str).glob('*.txt')
for path in pathlist:
    path_in_str = str(path)
    print(path_in_str)

    loader = TextLoader(path_in_str)
    documents = loader.load()
    docs = text_splitter.split_documents(documents)

    all_documents.extend(docs)

/content/text-files/star-method.txt
/content/text-files/resume-rules-for-swe.txt
/content/text-files/swe-bullet-points.txt
/content/text-files/xyz-method.txt
/content/text-files/common-mistakes.txt
/content/text-files/5-ats-myths-debunked.txt
/content/text-files/what-is-an-ats.txt
/content/text-files/perfective-from-sre-interviewer.txt
/content/text-files/effective-dev-resume.txt
/content/text-files/resume-format.txt
/content/text-files/ats-myths-busted.txt
/content/text-files/how-the-ats-actually-work.txt
/content/text-files/car-method.txt
/content/text-files/wiki.txt
/content/text-files/what-we-look-for-in-a-resume.txt


In [74]:
embedding = OpenAIEmbeddings()

In [118]:
libarary = FAISS.from_documents(all_documents, embedding)

In [119]:
query1 = "Tips for resume formatting"

answer = libarary.similarity_search(query1)

In [77]:
docs_and_scores = libarary.similarity_search_with_score(query1)

In [81]:
docs_and_scores

[(Document(id='e3433951-67ac-465c-92c0-52b5c1c89d40', metadata={'source': '/content/text-files/xyz-method.txt'}, page_content='1. Do the basic formatting.\n\nThe fundamental rule is to keep things simple when formatting and ensure \nthat your resume is readable. Make your resume simple and give it a \nprofessional look by using black text with consistent font size, style, \nand design elements to avoid distracting the recruiters. Format your \nresume as a pdf, and before hitting the submit button, check for typos, \nuse bullet points and make sure your contact info is up-to-date. \n\n2. Incorporate keyword phrases.'),
  0.1604591),
 (Document(id='e048d0ed-a910-4a2d-8a9d-37b89db06b56', metadata={'source': '/content/text-files/resume-format.txt'}, page_content="You'll also want to make sure that your resume uses a simple, basic \ndesign. Don't try to incorporate tables, text boxes, images, or unusual \nfonts. Many applicant tracking systems struggle to read those types of \ndesign elemen

In [129]:
retriever = libarary.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=retriever
  )

In [124]:
retriever_query = "How can I stand out when applying for jobs?"
results = qa.invoke(retriever_query)
print(results)

{'query': 'How can I stand out when applying for jobs?', 'result': ' One way to stand out when applying for jobs is to tailor your resume to the specific position you are applying for. This means addressing the job description and showcasing your skills and experiences that align with the job requirements. It is also important to add context to your resume and differentiate yourself by highlighting your accomplishments and problem-solving abilities. Additionally, bringing a physical copy of your resume on heavier paper to a traditional career fair can make it stand out and appear more professional.'}


In [122]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

loader = PyPDFLoader('/content/BangertBridgetResume-10.pdf')
pages = []
async for page in loader.alazy_load():
    pages.append(page)

documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    length_function=len
)

docs = text_splitter.split_documents(pages)
libarary = FAISS.from_documents(docs, embedding)

query1 = "What is Bridget's work experience"
answer = libarary.similarity_search(query1)

In [123]:
answer

[Document(id='356444d1-a2f3-4acd-a595-0106ca6a7a62', metadata={'producer': 'Skia/PDF m134 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'BangertBridgetResume', 'source': '/content/BangertBridgetResume-10.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Bridget  Bangert  bridgetmbangert@gmail.com |  g i t h u b . c o m / b r i d g e t b a n g e r t  |  l i n k e d i n . c o m / i n / b r i d g e t b a n g e r t   \nExperience  \n \nAccenture  \n \nData  Engineering  Senior  Analyst  June  2024  –  Present  ●  Enhanced  a  Kafka  ingest  framework  to  modernize  ETL  processes  of  over  15  source  systems  by  solutioning  a  \nmetadata-driven\n \ncolumn-mapping\n \nfunctionality\n \nto\n \nimprove\n \ndata\n \ntransformations\n \nand\n \nsystem\n \nscalability'),
 Document(id='032c9493-de4e-4d97-9d0a-4c4ac0530421', metadata={'producer': 'Skia/PDF m134 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'BangertBridgetR

In [133]:
retriever_query = "What company did Bridget work for?"

retriever = libarary.as_retriever()
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=retriever
  )
results = qa.invoke(retriever_query)
print(results)

{'query': 'What company did Bridget work for?', 'result': ' Bridget worked for Accenture.'}
